In [ ]:
class SIMgame:
    
    global playerName
    global compName
    global playerTurn
    global compNumber
    global playerNumber
    global winnerFound
    global movesCount
    global winner
    global movesPlayed
    global currentBoard
    
    movesPlayed = [[]for n in range(0,7)]
    
    for n in range(0,7):
        for m in range(0,7):
            movesPlayed[n].append(0)
    
    movesCount = 1
    compNumber = None
    playerNumber = None
    winner = 0
    winnerFound = False
    playerTurn = None
    compName = "AI"
    
    def getComputerMove():
        
        global playerName
        global compName
        global playerTurn
        global compNumber
        global playerNumber
        global winnerFound
        global movesCount
        global winner
        global movesPlayed
        global currentboard
        
        move = [None, None, None]
        startingDepth = movesCount
        currentBoardCopy = [[]for n in range(0,7)]
    
        for n in range(0,7):
            for m in range(0,7):
                currentBoardCopy[n].append(None)
                
        emptyPreviousMove = [None, None]
        emptyFirstMove = [None, None]
        
        #Part 1
        #Any move that is disjoint from first player's first move
        #For moves 1-3
        
        if (movesCount < 4):
            
            #Iterates through all moves
            for x in range(1,7):
                
                if playerTurn == True:
                    break
                    
                for y in range(x+1, 7):
                    
                    if playerTurn == True:
                        break
                        
                    #Check if move is disjoint
                    #If move has not been played
                    if movesPlayed[x][y] == 0:
                        
                        xDisjoint = True
                        yDisjoint = True
                        
                        #Check that no move is taken that includes x, if true at end of loop, continue to do the same for y
                        for y2 in range(1,7):
                            if movesPlayed[y2][x] != 0 or movesPlayed[x][y2] != 0:
                                xDisjoint = False
                                
                        #if xDisjoint is true at end of loop, continue to do the same for y
                        if xDisjoint == True:
                            for x2 in range(1,7):
                                if movesPlayed[y][x2] != 0 or movesPlayed[x2][y] == 0:
                                    yDisjoint == False
                                    
                        #If both x and y are disjointed, make this move and end turn
                        if xDisjoint == True and yDisjoint == True:
                            
                            #Choose x, y
                            movesPlayed[x][y] = compNumber
                            move[0] = x
                            move[1] = y
                            print(f"Turn {movesCount}: {compName}'s move is {move[0]} {move[1]}")
                            movesCount += 1
                            playerTurn = True
                            
            #From here, there is no disjoint move available or one has been made
            
            #If none was found
            if playerTurn == False:
                print("No disjointed move.")
                
        
        #Part 2 - Smart Heuristic for moves 4 through 6 (<7)
        elif movesCount < 7:
            
            move  = SIMgame.smartHeuristic()
            
            #Play chosen move
            movesPlayed[move[0]][move[1]] = compNumber
            print(f"Turn {movesCount}: {compName}'s move is {move[0]} {move[1]}")
            movesCount += 1
            playerTurn = True
            
        
        #If 15th move (last move), just choose the last available move
        elif movesCount == 15:
            
            validMove = False
            
            for x in range(1,7):
                if validMove == False:
                    
                    for y in range(x+1, 7):
                        if validMove == False:
                            
                            if movesPlayed[x][y] == 0:
                                
                                movesPlayed[x][y] = compNumber
                                validMove = True
                                print(f"Turn {movesCount}: {compName}'s move is {x} {y}")
                                movesCount += 1
                                playerTurn = True
                                
                                
        #Otherwise minimax
        else:
            
            #Copy current board first
            for i in range(0, 7):
                for j in range(0,7):
                    currentBoardCopy[i][j] = movesPlayed[i][j]
                    
            move = SIMgame.minimax(True, startingDepth, currentBoardCopy, emptyPreviousMove, emptyFirstMove)
            
            #If there is no path that guarantees minimax a win, and the best score it can do is -1:
            #Use the smart heuristic method instead, to avoid minimax giving up early since
            #it is equally likely to choose a move that loses immediately and a move that loses later in the game
            if move[2] == -1:
                
                move = SIMgame.smartHeuristic()
                
            #Minimax will return 0,0 if it has no available moves left without losing
            #So if it returns 0, play the first available move
            if move[0] != 0:
                movesPlayed[move[0]][move[1]] = compNumber
                print(f"Turn {movesCount}: {compName}'s move is {move[0]} {move[1]}")
                movesCount += 1
                playerTurn = True
                
            #If it does return 0, play the first available move and lose (greedy)
            else:
                validMove = False
                
                for x in range(1,7):
                    if validMove == False:
                        
                        for y in range(x+1, 7):
                            if validMove == False:
                                
                                if movesPlayed[x][y] == 0:
                                    movesPlayed[x][y] = compNumber
                                    validMove = True
                                    print(f"Turn {movesCount}: {compName}'s move is {x} {y}")
                                    movesCount += 1
                                    playerTurn = True
       
    
    #SMART HEURISTIC FUNCTION
    #Will assign moves positive and negative points based on quality, then select the best one
    #Points are assigned in 2 index of move[]
    def smartHeuristic():
        
        global playerName
        global compName
        global playerTurn
        global compNumber
        global playerNumber
        global winnerFound
        global movesCount
        global winner
        global movesPlayed
        global currentboard
        
        move = [None, None, None]
        
        move[2] = -10000
        
        #Iterates through all moves
        for x in range(1,7):
            for y in range(x+1, 7):
                score = -10000
                
                #Legal move check
                #Score becomes 0 if move is legal
                if movesPlayed[x][y] == 0:
                    score = score + 10000
                    
                
                #Loss check (AI-AI-AI)
                #Check all y2 values for the current x (row)
                for y2 in range(x+1, 7):
                    
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x][y2] == compNumber:
                        if movesPlayed[y][y2] == compNumber or movesPlayed[y2][y] == compNumber:
                            score = score - 100
                
                #Check all x2 values for the current y (column)
                for x2 in range(y-1, 0, -1):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x2][y] == playerNumber:
                        if movesPlayed[x][x2] == playerNumber or movesPlayed[x2][x] == playerNumber:
                            score = score - 100
                            
                #Closing Player Triangle Check
                #AI-Player-Player Triangle
                #Check all y2 values for the current x (row)
                for y2 in range(x+1, 7):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x][y2] == playerNumber:
                        if movesPlayed[y][y2] == playerNumber or movesPlayed[y2][y] == playerNumber:
                            score = score - 10
                
                #Check all x2 values for the current y (column)
                for x2 in range(y-1, 0, -1):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x2][y] == playerNumber:
                        if movesPlayed[x][x2] == playerNumber or movesPlayed[x2][x] == playerNumber:
                            score = score - 10
                            
                
                #Partial Triangle Check (Player check first)
                #AI-AI-Player (checking for 1 AI and 1 Player move)
                #Check all y2 values for the current x (row)
                for y2 in range(x+1, 7):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x][y2] == playerNumber:
                        if movesPlayed[y][y2] == compNumber or movesPlayed[y2][y] == compNumber:
                            score = score + 5
                            
                #Check all x2 values for the current y (column)
                for x2 in range(y-1, 0, -1):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x2][y] == playerNumber:
                        if movesPlayed[x][x2] == compNumber or movesPlayed[x2][x] == compNumber:
                            score = score + 5
                            
                #Partial Triangle Check (AI check first)
                #AI-AI-Player (checking for 1 AI and 1 Player move)
                #Check all y2 values for the current x (row)
                for y2 in range(x+1, 7):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x][y2] == compNumber:
                        if movesPlayed[y][y2] == playerNumber or movesPlayed[y2][y] == playerNumber:
                            score = score + 5
                            
                #Check all x2 values for the current y (column)
                for x2 in range(y-1, 0, -1):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x2][y] == compNumber:
                        if movesPlayed[x][x2] == playerNumber or movesPlayed[x2][x] == playerNumber:
                            score = score + 5
                            
                #AI-AI Connection
                #Check all y2 values for the current x (row)
                for y2 in range(x+1, 7):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x][y2] == compNumber:
                        if movesPlayed[y][y2] == 0 or movesPlayed[y2][y] == 0:
                            score = score - 5
                            
                #Check all x2 values for the current y (column)
                for x2 in range(y-1, 0, -1):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x2][y] == compNumber:
                        if movesPlayed[x][x2] == 0 or movesPlayed[x2][x] == 0:
                            score = score - 5
                            
                #Player-AI Connection
                #Check all y2 values for the current x (row)
                for y2 in range(x+1, 7):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x][y2] == playerNumber:
                        if movesPlayed[y][y2] == 0 or movesPlayed[y2][y] == 0:
                            score = score + 1
                            
                #Check all x2 values for the current y (column)
                for x2 in range(y-1, 0, -1):
                    #Check any moves with matching x or y, to see if their other coordinate matches
                    if movesPlayed[x2][y] == playerNumber:
                        if movesPlayed[x][x2] == 0 or movesPlayed[x2][x] == 0:
                            score = score + 1
                            
                #Finally, if move score is better than the stored score, replace stored move
                if score > move[2]:
                    move[0] = x
                    move[1] = y
                    move[2] = score
                    
        return move
    
    
    #MINIMAX FUNCTION
    def minimax (maximizingPlayer, depth, currentBoard, previousMove, firstMove):
        
        global playerName
        global compName
        global playerTurn
        global compNumber
        global playerNumber
        global winnerFound
        global movesCount
        global winner
        global movesPlayed
        global currentboard
        
        #Index 0 is move x, index 1 is move y, index 2 is score (0, 1, -1))
        output = [None, None, None]
        
        #Generate all possible moves
        possibleMoves = [[None, None] for n in range(0, (16-depth))]
        index = 0
        
        for x in range(1, 7):
            for y in range(x+1, 7):
                #If move is possible and has not been made, add to the array of possible moves
                if currentBoard[x][y] == 0:
                    possibleMoves[index][0] = x
                    possibleMoves[index][1] = y
                    index += 1
                    
        #Setting first move to be passed down to all child nodes - only happens on SECOND call
        if depth == movesCount + 1:
            firstMove[0] = previousMove[0]
            firstMove[1] = previousMove[1]
            
        #If it is the maximizing player's turn, return the minimax resulting in the highest score:
        if maximizingPlayer == True:
            currentMax = [None, None, None]
            #NOTE - if returned score is -999, that means that there were no more nodes to test
            currentMax[0] = 0
            currentMax[1] = 0
            currentMax[2] = -999
            #Lower than all scores, will ensure that score is set to next highest encountered
            
            #Iterates through all valid moves
            for x in range(0, index):
                
                #Update board for new move
                updatedBoard = [[]for n in range(0,7)]
    
                for n in range(0,7):
                    for m in range(0,7):
                        updatedBoard[n].append(None)
                        
                #Copy currentBoard first
                for i in range(0,7):
                    for j in range(0,7):
                        updatedBoard[i][j] = currentBoard[i][j]
                        
                #Now add new move
                updatedBoard[possibleMoves[x][0]][possibleMoves[x][1]] = compNumber
                
                #If leaf node:
                #If move will result in a win or loss (using score function), check against current max
                if SIMgame.countTriangles(updatedBoard) == -1 or SIMgame.countTriangles(updatedBoard) == 1:
                    #If score is more than currentMax, reassign this move and score to currentMax
                    if SIMgame.countTriangles(updatedBoard) > currentMax[2]:
                        currentMax[0] = possibleMoves[x][0]
                        currentMax[1] = possibleMoves[x][1]
                        currentMax[2] = SIMgame.countTriangles(updatedBoard)
                        
                #If a move does not result in a win or loss (not a leaf node),
                #NEED TO RUN MINIMAX AGAIN, and check if minimax result is better than currentMax
                if SIMgame.countTriangles(updatedBoard) == 0:
                    
                    minimaxResult = [None, None, None]
                    
                    minimaxResult = SIMgame.minimax(False, depth + 1, updatedBoard, possibleMoves[x], firstMove)
                    
                    #Checking if the result is better than currentMax, setting new currentMax if it is
                    if minimaxResult[2] > currentMax[2]:
                        
                        currentMax[0] = possibleMoves[x][0]
                        currentMax[1] = possibleMoves[x][1]
                        currentMax[2] = minimaxResult[2]
                        
                    
            #If there were no options to test, error check
            if currentMax[2] == -999:
                print("Error")
                
            #Return score of currentMax with firstMove move
            output[0] = currentMax[0]
            output[1] = currentMax[1]
            output[2] = currentMax[2]
            return output
        
        
        #If it is the minimizing player's turn:
        if maximizingPlayer == False:
            currentMin = [None, None, None]
            currentMin[0] = 0
            currentMin[1] = 0
            currentMin[2] = 999
            #Higher than all scores, will ensure that score is set to next lowest encountered
            
            #Iterates through all valid moves
            for x in range(0, index):
                
                #Update board for new move
                updatedBoard = [[]for n in range(0,7)]
    
                for n in range(0,7):
                    for m in range(0,7):
                        updatedBoard[n].append(None)
                        
                #Copy currentBoard first
                for i in range(0,7):
                    for j in range(0,7):
                        updatedBoard[i][j] = currentBoard[i][j]
                        
                #Now add new move
                updatedBoard[possibleMoves[x][0]][possibleMoves[x][1]] = playerNumber
                
                #If leaf node:
                #If move will result in a win or loss (using score function), check against current min
                if SIMgame.countTriangles(updatedBoard) == -1 or SIMgame.countTriangles(updatedBoard) == 1:
                    # If score is less than currentMin, reassign this move and score to currentMin
                    if SIMgame.countTriangles(updatedBoard) < currentMin[2]:
                        currentMin[0] = possibleMoves[x][0]
                        currentMin[1] = possibleMoves[x][1]
                        currentMin[2] = SIMgame.countTriangles(updatedBoard)
                        
                #If a move does not result in a win or loss (not a leaf node),
                #NEED TO RUN MINIMAX AGAIN, and check if minimax result is less than currentMin
                if SIMgame.countTriangles(updatedBoard) == 0:
                    
                    minimaxResult = SIMgame.minimax(True, depth + 1, updatedBoard, possibleMoves[x], firstMove)
                    
                    #Checking if the result is less than currentMin, setting new currentMin if it is
                    if minimaxResult[2] < currentMin[2]:
                        currentMin[0] = possibleMoves[x][0]
                        currentMin[1] = possibleMoves[x][1]
                        currentMin[2] = minimaxResult[2]
                    
                    
            #If there were no options to test, error check
            if currentMin[2] == 999:
                print("Error")
                
            #Return score of currentMin with firstMove move
            output[0] = currentMin[0]
            output[1] = currentMin[1]
            output[2] = currentMin[2]
            return output
        
        return None
    
    
    #SCORING FUNCTION
    def countTriangles(currentBoard):
        
        score = 0
        
        redBlue = [0, 0, 0]
        
        for x in range(1,7):
            for y in range(x+1, 7):
                for z in range(y+1, 7):
                    
                    if currentBoard[x][y] == 1 and currentBoard[x][z] == 1 and currentBoard[y][z] == 1:
                        redBlue[1] += 1
                        
                    if currentBoard[x][y] == 2 and currentBoard[x][z] == 2 and currentBoard[y][z] == 2:
                        redBlue[2] += 1
                        
        if compNumber == 1:
            
            if redBlue[1] > 0:
                score = -1
                
            if redBlue[2] > 0:
                score = 1
                
        if compNumber == 2:
            
            if redBlue[1] > 0:
                score = 1
                
            if redBlue[2] > 0:
                score = -1
        
        return score
    
    
    #Move entry system - by Richard Hoshino
    def getPlayerMove():
        
        global playerName
        global compName
        global playerTurn
        global compNumber
        global playerNumber
        global winnerFound
        global movesCount
        global winner
        global movesPlayed
        global currentBoard
        
        validMove = False
        
        while validMove == False:
            
            print(f"Turn {movesCount}: Input your next move: ")
            x = int(input())
            y = int(input())
            
            if x > 0 and x <= 6 and y > 0 and y <= 6 and x < y and movesPlayed[x][y] == 0:
                validMove = True
                movesPlayed[x][y] = playerNumber
                movesCount += 1
                playerTurn = False
                
            else:
                print("Illegal move: try again")
                
                
    def checkWinner():
        
        global playerName
        global compName
        global playerTurn
        global compNumber
        global playerNumber
        global winnerFound
        global movesCount
        global winner
        global movesPlayed
        global currentboard
        
        redBlue = [0, 0, 0]
        
        for x in range(1,7):
            for y in range(x+1, 7):
                for z in range(y+1, 7):
                    
                    if movesPlayed[x][y] == 1 and movesPlayed[x][z] == 1 and movesPlayed[y][z] == 1:
                        redBlue[1] += 1
                        
                    if movesPlayed[x][y] == 2 and movesPlayed[x][z] == 2 and movesPlayed[y][z] == 2:
                        redBlue[2] += 1
                        
        if redBlue[1] > 0:
            winnerFound = True
            winner = 2
            
        if redBlue[2] > 0:
            winnerFound = True
            winner = 1
            
            
    def main():
        
        global playerName
        global compName
        global playerTurn
        global compNumber
        global playerNumber
        global winnerFound
        global movesCount
        global winner
        global movesPlayed
        global currentBoard
        
        print("Let's play the Triangle Game. What is your name? ")
        playerName = input()
        
        print(f"True or False: will {playerName} be the first player? ")
        playerTurn = input()
        if playerTurn == "True" or playerTurn == "true":
            playerTurn = True
        elif playerTurn == "False" or playerTurn == "false":
            playerTurn = False
        else:
            playerTurn = True
        
        if playerTurn == True:
            compNumber = 2
            playerNumber = 1
            
        else:
            compNumber = 1
            playerNumber = 2
            SIMgame.getComputerMove()
            
        while winnerFound == False and movesCount <= 15:
            
            if playerTurn == True:
                SIMgame.getPlayerMove()
                SIMgame.checkWinner()
            else:
                SIMgame.getComputerMove()
                SIMgame.checkWinner()
                
        print("")
        if winner == 0:
            print("The game ends in a tie.")
        if winner == playerNumber:
            print(f"{playerName} wins")
        if winner == compNumber:
            print(f"{compName} wins")


In [ ]:
SIMgame.main()